## Imports

In [58]:
import os
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import requests
import lxml
from lxml.html.soupparser import fromstring
import prettify
import numbers
import htmltext
import urllib
from requests_html import HTML
from requests_html import HTMLSession
import json
from openpyxl import load_workbook
from openpyxl.styles import Font, Color, colors, fills

OurFilePath = '/Users/homayoonshobeiri/Desktop/Rental Property Investment/Rental_Property_Search.xlsx'

import warnings
warnings.filterwarnings('ignore')

## Creating the Link for all the pages in a search criteria

In [37]:
def get_pages_links(city,state,zipcode,criteria,num_pages):
    # criteria (sale,rental,sold)
    # num_page: number of pages in our search result input manually by checking the url
    # all inputs except num_pages are strings
    
    list_pages = []
    for i in range(1,num_pages+1):
        if i == 1:
            url = 'https://www.zillow.com/{0}-{1}-{2}/{3}/'.format(city,state,zipcode,criteria)
        else:
            url = 'https://www.zillow.com/{0}-{1}-{2}/{3}/{4}_p'.format(city,state,zipcode,criteria,i)
        
        list_pages.append(url)
            
    return(list_pages)

## Getting all the property links in each page from get_pages_links() function

In [140]:
def get_raw_data(city,state,zipcode,criteria,num_pages):
    # criteria in {sold,sale,rentals}
    req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
        }
    
    list_page = get_pages_links(city,state,zipcode,criteria,num_pages)
    print(list_page)
    list_property_links = []
    
    for url in list_page:
        with requests.Session() as s:
            r = s.get(url,headers = req_headers)
            soup = BeautifulSoup(r.content,'html.parser')
            num_class_tags = len(soup.find_all(class_ = "list-card-link"))
            link = soup.find_all('script',{'type':"application/json"})
            list_property_links.append(link)
                
    return(list_property_links)


def intermediary_cleaned_data(raw_data):
    
    listing_dict = list()
    num_pages = len(raw_data)
    for i in range(num_pages):
        raw_listing_data = list(raw_data[i][1])
        step1_cleaned_data = raw_listing_data[0].replace("<!--",'')
        step2_cleaned_data = step1_cleaned_data.replace("-->",'')
        dict_list_data = json.loads(step2_cleaned_data)
        final_dict_list_data = dict_list_data['cat1']['searchResults']['listResults']
        
        n = len(final_dict_list_data)
        for j in range(n):
            listing_dict.append(final_dict_list_data[j])
            
    
    listing_df = pd.DataFrame(listing_dict)
    
    return(listing_df)
    
    
def final_cleaned_data(inter_clean_data,max_price = 250000):
    
    data1 = inter_clean_data.drop(['zpid', 'id', 'providerListingId', 'imgSrc', 'hasImage','countryCurrency', 'price','isUndisclosedAddress','badgeInfo','isSaved', 'isUserClaimingOwner',
       'isUserConfirmedClaim', 'pgapt', 'sgapt','shouldShowZestimateAsPrice', 'has3DModel', 'hasVideo', 'isHomeRec','hasAdditionalAttributions', 'isFeaturedListing', 'availabilityDate',
       'list', 'relaxed'],1)
    
    # 'streetViewMetadataURL', 'streetViewURL','lotAreaString',,'openHouseDescription'
    data2 = data1.rename(columns={'unformattedPrice':'Price'})
    data3 = pd.concat([data2.drop(['latLong'],axis=1),data2.latLong.apply(pd.Series)],axis=1)
    
    Days_on_zillow = data3.variableData.apply(pd.Series)['text']
    data4 = pd.concat([data3.drop(['variableData'],axis=1),Days_on_zillow],axis=1)
    data5 = data4.rename(columns={'text':'DaysOnZillow'})
    data5['DaysOnZillow'] = data5['DaysOnZillow'].map(lambda x: x.strip('days on Zillow'))
    data5['DaysOnZillow'] = data5['DaysOnZillow'].apply(lambda x: int(x) if len(x) <= 3 else 0) 

    
    hdp_exploded = data5.hdpData.apply(pd.Series)['homeInfo'].apply(pd.Series)
    data6 = pd.concat([data5.drop(['hdpData'],axis=1),hdp_exploded[['homeType','rentZestimate','priceChange','isNonOwnerOccupied','isPreforeclosureAuction',
                                                                    'isPremierBuilder','taxAssessedValue','lotAreaValue','lotAreaUnit','listing_sub_type']]],axis=1)
    
    listing_sub_type_exploded = data6.listing_sub_type.apply(pd.Series)
    data7 = pd.concat([data6.drop(['listing_sub_type'],axis=1),listing_sub_type_exploded],axis=1)
    
    data8 = data7.drop(['statusText'],axis=1)
    
    data9 = data8.rename(columns={'area':'bldSQFT'})
    
    def calc_lotArea(s):
        if (s['lotAreaUnit'] == 'sqft'):
            return s['lotAreaValue']
        else:
            return s['lotAreaValue'] * 43560
    
    data9['lotSQFT'] = data9.apply(calc_lotArea,axis=1)
    data10 = data9.drop(['lotAreaValue','lotAreaUnit'],axis=1)
    
    data11 = data10[data10['Price'] <= max_price]
    
    data12 = data11[data11['homeType'] == 'SINGLE_FAMILY']
    
    
    
    
    return data12

In [142]:
#a = get_raw_data('houston','tx','77077','sale',1)
#b = intermediary_cleaned_data(a)
#c = final_cleaned_data(b)

#nonNull_arrays = []
#for colNames in c.columns:
#    n = len(c[colNames].isna())
#    nonNull_arrays.append(n)
    
#nonNull_arrays

#c['rentZestimate'].isna()

a1 = get_raw_data('houston','tx','77084','sale',2)
b1 = intermediary_cleaned_data(a1)
c1 = final_cleaned_data(b1)



['https://www.zillow.com/houston-tx-77084/sale/', 'https://www.zillow.com/houston-tx-77084/sale/2_p']


### write all the props based on zip to our Rental_Property_search.xlsx'

In [146]:
def write_to_our_search(df,FilePath = OurFilePath):
    
    df_Our_Order = df[['detailUrl','statusType','homeType','address','Price','beds','baths','bldSQFT','isZillowOwned','zestimate','brokerName',
                       'builderName','latitude','longitude','DaysOnZillow','rentZestimate','priceChange','isNonOwnerOccupied','isPreforeclosureAuction','isPremierBuilder',
                      'taxAssessedValue','lotSQFT','is_FSBA','is_newHome']]
    ExcelWorkbook = load_workbook(FilePath)
    writer = pd.ExcelWriter(FilePath, engine = 'openpyxl')
    writer.book = ExcelWorkbook
    zipCode = df.addressZipcode[0]
    
    if zipCode in ExcelWorkbook.sheetnames:
        del ExcelWorkbook[zipCode]
    
    #df_Our_Order.loc['Average'] = df_Our_Order.mean()
    
    df_Our_Order.to_excel(writer,sheet_name = zipCode,na_rep='NULL',header=True,index=False)
    #df_Our_Order_Avg_row = df_Our_Order.iloc[-1]
    #df_Our_Order_Avg_row.fill = fills.PatternFill(patternType = 'solid', fgColor=Color(rgb='00FF00'))
    
    writer.save()
    writer.close()

In [147]:
write_to_our_search(c1)

In [163]:
req_headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.8',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
        }

with requests.Session() as s:
        r = s.get('https://www.har.com/zipcode_77064/realestate/for_sale',headers = req_headers)
        soup = BeautifulSoup(r.content,'html.parser')
            

In [164]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/><title>77064 Homes for Sale - HAR.com</title>
<meta content="69 Homes for Sale in Houston, TX 77064. Browse photos, see new properties and research neighborhoods on HAR.com" name="description"/>
<meta content="HAR, 77064 real estate, 77064 homes, 77064 Homes for Sale, 77064 properties, 77064 listings" name="keywords"/>
<link href="https://www.har.com/zipcode_77064/realestate/for_sale" rel="canonical"/>
<meta content="HRIS, Inc." name="author">
<meta content="Copyright (c) 1997-2022 HRIS, Inc." name="copyright">
<meta content="77064 Homes for Sale - HAR.com" property="og:title"/>
<link href="https://www.har.com/resources/images/common/favicon_32.png" rel="shortcut icon" sizes="32x32"/>
<link href="https://www.har.com/resources/images/common/favicon_48.png" rel="shortcut icon" sizes="48x48"/>
<link href="https